In [1]:
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import json

In [2]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from models import *
from torch.autograd import Variable
from new.new_rotation_dataset import NewRotationDataset
import numpy as np
import math
import utils


In [3]:
parser = argparse.ArgumentParser(description='Rotation Training with Scores')
parser.add_argument('--data-dir', type=str, default="bedroom", metavar='S')
parser.add_argument('--num-workers', type=int, default=6, metavar='N')
parser.add_argument('--last-epoch', type=int, default=-1, metavar='N')
parser.add_argument('--train-size', type=int, default=6400, metavar='N')
parser.add_argument('--save-dir', type=str, default="train/bedroom", metavar='S')
parser.add_argument('--ablation', type=str, default=None, metavar='S')
parser.add_argument('--lr', type=float, default=0.001, metavar='N')
parser.add_argument('--eps', type=float, default=1e-6, metavar='N')
args = parser.parse_args("")

In [4]:

save_dir = args.save_dir
utils.ensuredir(save_dir)
learning_rate = args.lr
batch_size = 16

In [5]:
num_categories = len(json.load(open("new/valid_types.json")))

In [6]:
if args.ablation is None:
    num_input_channels = num_categories+9
elif args.ablation == "basic":
    num_input_channels = 7
elif args.ablation == "depth":
    num_input_channels = 2
else:
    raise NotImplementedError

logfile = open(f"{save_dir}/log_rotation.txt", 'w')
def LOG(msg):
    print(msg)
    logfile.write(msg + '\n')
    logfile.flush()

In [7]:
LOG('Building model...')
model = resnet101(num_classes=2, num_input_channels=num_input_channels)
loss = nn.CrossEntropyLoss()
softmax = nn.Softmax(dim=1)

LOG('Converting to CUDA...')
model.cuda()
loss.cuda()
softmax.cuda()

Building model...
Converting to CUDA...


Softmax(dim=1)

In [8]:
args.data_dir = "/home/ubuntu/research/3D-FRONT-ToolBox/metadata43DSLN/all_room_info_b.json"

In [9]:
LOG('Building dataset...')
train_dataset = NewRotationDataset(
    data_dir = args.data_dir
)
#Size of validation set is 160 by default
validation_dataset = NewRotationDataset(
    data_dir = args.data_dir
)

Building dataset...


In [10]:
LOG('Building data loader...')
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size = batch_size,
    num_workers = args.num_workers,
    shuffle = True
)

validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size = batch_size,
    num_workers = 0,
    shuffle = True,
)

Building data loader...


In [11]:
LOG('Building optimizer...')
optimizer = optim.Adam(model.parameters(),
    lr = learning_rate,
    betas = (0.9,0.999),
    eps = args.eps
)


Building optimizer...


In [12]:
if args.last_epoch < 0:
    load = False
    starting_epoch = 0
else:
    load = True
    last_epoch = args.last_epoch

if load:
    LOG('Loading saved models...')
    model.load_state_dict(torch.load(f"{save_dir}/rotation_{last_epoch}.pt"))
    optimizer.load_state_dict(torch.load(f"{save_dir}/rotation_optim_backup.pt"))
    starting_epoch = last_epoch + 1

In [13]:
current_epoch = starting_epoch
num_seen = 0

model.train()
LOG(f'=========================== Epoch {current_epoch} ===========================')

=========================== Epoch 0 ===========================


In [14]:
def validate():
    model.eval()
    total_loss = []
    total_accuracy = []
    for _, (data, target) in enumerate(validation_loader):
        with torch.no_grad():
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = model(data)
            loss_val = loss(output, target)
            total_loss.append(loss_val.item())

            output = softmax(output)
            outputs = output.cpu().data.numpy()
            targets = target.cpu().data.numpy()
            predictions = np.argmax(outputs, axis=1)
            num_correct = np.mean(predictions == targets)
            total_accuracy.append(num_correct)

    LOG(f'Loss: {np.mean(total_loss)}, Accuracy: {np.mean(total_accuracy)}')

In [15]:
total_epoch = 20

In [16]:
for ep in range(total_epoch):
    print("Epoch:", ep)
    train_loss_list = []
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss_val = loss(output, target)
        loss_val.backward()
        optimizer.step()
        
        train_loss_list.append(loss_val.item())

        num_seen += batch_size
        if num_seen % 800 == 0:
            LOG(f'Examples {num_seen}/10000 Train Loss {np.mean(train_loss_list)}')
            train_loss_list.clear()
            
        if num_seen % 10000 == 0:
            LOG('Validating')
            validate()
            model.train()
            num_seen = 0
            current_epoch += 1
            LOG(f'=========================== Epoch {current_epoch} ===========================')
            if current_epoch % 10 == 0:
                torch.save(model.state_dict(), f"{save_dir}/rotation_{current_epoch}.pt")
                torch.save(optimizer.state_dict(), f"{save_dir}/rotation_optim_backup.pt")

Epoch: 0


0it [00:00, ?it/s]

Examples 800/10000 Train Loss 0.7403677624464035
Epoch: 1


0it [00:00, ?it/s]

Examples 1600/10000 Train Loss 0.5504690277225831
Epoch: 2


...

Examples 8000/10000 Train Loss 0.5198609232902527
Examples 8800/10000 Train Loss 0.4887677624821663
Epoch: 18


0it [00:00, ?it/s]

Examples 9600/10000 Train Loss 0.4016161355617884
Validating
Loss: 0.38362583472873224, Accuracy: 0.8186026936026937
=========================== Epoch 2 ===========================
Epoch: 19


0it [00:00, ?it/s]

Examples 800/10000 Train Loss 0.3919739091525907
